In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, lit
from delta.tables import DeltaTable
import os

In [2]:
# Define MinIO settings from environment variables
MINIO_HOST = os.environ.get("MINIO_HOST")
AWS_ACCESS_KEY_ID = "minio"
AWS_SECRET_ACCESS_KEY = "minio-password"

# Get Spark Master URL from environment or use default
SPARK_MASTER_URL = os.environ.get("SPARK_MASTER_URL", "spark://spark-master-service:7077")

In [3]:
# Create Spark Session with Delta Lake and MinIO configurations
spark = SparkSession.builder \
    .appName("Delta Lake Pipeline") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_HOST) \
    .config("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY_ID) \
    .config("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:3.2.0") \
    .getOrCreate()

In [4]:
# Base path for the Delta tables in the MinIO bucket
base_path = "s3a://datalakehouse/deltalake/"

In [5]:
# --- Bronze Layer: Ingest raw data ---
# Simulate new raw data coming in as a Spark DataFrame
print("--- Ingesting raw data to Bronze layer ---")
raw_data = spark.createDataFrame(
    [
        (1, "Alice", "New York", "2025-01-01"),
        (2, "Bob", "Los Angeles", "2025-01-02"),
        (3, "Charlie", "Chicago", "2025-01-03")
    ],
    ["id", "name", "location", "timestamp"]
)


--- Ingesting raw data to Bronze layer ---


In [6]:
bronze_path = f"{base_path}bronze/users/"
raw_data.write.format("delta").mode("overwrite").save(bronze_path)
print("Bronze table created/updated.")


Bronze table created/updated.


In [7]:
# --- Silver Layer: Clean and enrich data ---
# Read the Bronze table
print("\n--- Processing data for Silver layer ---")
bronze_df = spark.read.format("delta").load(bronze_path)

bronze_df.show()


--- Processing data for Silver layer ---
+---+-------+-----------+----------+
| id|   name|   location| timestamp|
+---+-------+-----------+----------+
|  3|Charlie|    Chicago|2025-01-03|
|  2|    Bob|Los Angeles|2025-01-02|
|  1|  Alice|   New York|2025-01-01|
+---+-------+-----------+----------+



In [8]:
# Perform a simple cleaning/enrichment step
silver_df = bronze_df.withColumn("name", col("name").alias("user_name")).drop("location")

silver_path = f"{base_path}silver/users/"

silver_df.show()

+---+-------+----------+
| id|   name| timestamp|
+---+-------+----------+
|  3|Charlie|2025-01-03|
|  2|    Bob|2025-01-02|
|  1|  Alice|2025-01-01|
+---+-------+----------+



In [9]:
# Use MERGE for upserting data based on a key
if DeltaTable.isDeltaTable(spark, silver_path):
    delta_table = DeltaTable.forPath(spark, silver_path)
    delta_table.alias("target") \
        .merge(silver_df.alias("source"), "target.id = source.id") \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
    print("Silver table updated with a MERGE operation.")
else:
    silver_df.write.format("delta").mode("overwrite").save(silver_path)
    print("Silver table created.")


silver_df.show()

Silver table updated with a MERGE operation.
+---+-------+----------+
| id|   name| timestamp|
+---+-------+----------+
|  3|Charlie|2025-01-03|
|  2|    Bob|2025-01-02|
|  1|  Alice|2025-01-01|
+---+-------+----------+



In [10]:
# --- Gold Layer: Aggregate for analytics ---
# Read the Silver table
print("\n--- Aggregating data for Gold layer ---")
silver_df = spark.read.format("delta").load(silver_path)


--- Aggregating data for Gold layer ---


In [11]:
# Perform a simple aggregation
gold_df = silver_df.groupBy("name").count().alias("user_count")

gold_path = f"{base_path}gold/user_summary/"
gold_df.write.format("delta").mode("overwrite").save(gold_path)
print("Gold table created/updated.")

gold_df.show()


Gold table created/updated.
+-------+-----+
|   name|count|
+-------+-----+
|Charlie|    1|
|    Bob|    1|
|  Alice|    1|
+-------+-----+



In [12]:
# --- Time Travel and Query Gold Layer ---
print("\n--- Reading Gold table with Time Travel ---")
gold_table = DeltaTable.forPath(spark, gold_path)
gold_history = gold_table.history()
gold_history.show(truncate=False)


--- Reading Gold table with Time Travel ---
+-------+-------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp          |userId|userName|operation|operationParameters                   |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                          |userMetadata|engineInfo                         |
+-------+-------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------+------------+-----------------------------------+
|1      |2025-09-29 14:20:54|NULL  |NULL    |WRITE    |{mode -> Overwrite, partitionBy -> []}|NULL|NULL    |NULL     |0          |Serializable  |false 

In [13]:
# Read the latest version
print("Reading latest gold table:")
spark.read.format("delta").load(gold_path).show()

Reading latest gold table:
+-------+-----+
|   name|count|
+-------+-----+
|Charlie|    1|
|    Bob|    1|
|  Alice|    1|
+-------+-----+



In [14]:
# Read a previous version (if history exists)
if gold_history.count() > 1:
    old_version = gold_history.collect()[1]["version"]
    print(f"Reading an older version ({old_version}) of the gold table:")
    spark.read.format("delta").option("versionAsOf", old_version).load(gold_path).show()

spark.stop()

Reading an older version (0) of the gold table:
+-------+-----+
|   name|count|
+-------+-----+
|Charlie|    1|
|  Alice|    1|
|    Bob|    1|
+-------+-----+

